In [1]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import datetime

Cargo los dataframes originales

In [2]:
df_original = pd.read_csv('events_up_to_01062018.csv', low_memory = False)
df_labels= pd.read_csv('labels_training_set.csv', low_memory = False)

Cargo los features numericos previamente encontrados en el notebook *jp_features*

In [3]:
df_features_numericos = pd.read_csv('features_numericos.csv', low_memory=False)

In [4]:
df_features_numericos = df_features_numericos.iloc[:,:15]

In [5]:
df_ingresos_diarios = df_original[['person','timestamp','event']]

In [6]:
df_ingresos_diarios['timestamp'] = pd.to_datetime(df_ingresos_diarios['timestamp'])

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df_promedio_ingresos_diarios = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('d')]).size().reset_index(name='visitas_diarias')

In [8]:
df_promedio_ingresos_diarios = df_promedio_ingresos_diarios.groupby('person')['visitas_diarias'].mean().reset_index(name='promedio_registros_diarias')

In [9]:
df_promedio_ingresos_diarios.head()

,person,promedio_registros_diarias
0,0008ed71,6.000000
1,00091926,20.363636
2,00091a7a,10.000000
3,000ba417,68.666667
4,000c79fe,17.000000


In [10]:
df_dias_ingresando = df_ingresos_diarios
df_dias_ingresando['date'] = df_dias_ingresando['timestamp'].dt.date

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
df_dias_ingresando = df_dias_ingresando.drop_duplicates(['person','date'])

In [12]:
df_dias_ingresando = df_dias_ingresando.groupby('person').size().reset_index(name='dias_ingresando')

In [13]:
df_promedio_ingresos_diarios = df_promedio_ingresos_diarios.merge(df_dias_ingresando, on='person', how='left')

In [14]:
df_promedio_ingresos_diarios.head()

,person,promedio_registros_diarias,dias_ingresando
0,0008ed71,6.000000,1
1,00091926,20.363636,22
2,00091a7a,10.000000,1
3,000ba417,68.666667,3
4,000c79fe,17.000000,1


In [15]:
df_promedio_ingresos_por_hora = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('h')]).size().reset_index(name='visitas_por_hora')

In [16]:
df_promedio_ingresos_por_hora = df_promedio_ingresos_por_hora.groupby('person')['visitas_por_hora'].mean().reset_index(name='promedio_visitas_hora')

In [17]:
df_promedio_ingresos_diarios = df_promedio_ingresos_diarios.merge(df_promedio_ingresos_por_hora, on='person', how='left')

In [18]:
df_promedio_ingresos_diarios.shape

(38829, 4)

In [19]:
df_features_numericos = df_features_numericos.merge(df_promedio_ingresos_diarios, on='person', how='left')

In [20]:
df_features_numericos.head()

,person,Cantidad_visitas,Promedio_visitas_producto,horas_mirando_productos,visitas_mes1,visitas_mes2,visitas_mes3,visitas_mes4,visitas_mes5,promedio_ingreso_mensual,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5,promedio_registros_diarias,dias_ingresando,promedio_visitas_hora
0,4886f805,4.0,5.043919,0.315278,0.0,0.0,0.0,0.0,9.0,1.8,NaN,NaN,9.0,NaN,NaN,9.000000,1,9.000000
1,ad93850f,17.0,9.904207,22.058056,0.0,0.0,0.0,0.0,65.0,13.0,NaN,13.0,45.0,7.0,NaN,16.250000,4,13.000000
2,0297fc1e,213.0,9.325721,191.342500,120.0,48.0,87.0,139.0,173.0,113.4,67.0,99.0,151.0,200.0,50.0,9.610169,59,5.906250
3,2d681dd8,6.0,8.860133,0.092222,0.0,0.0,0.0,0.0,26.0,5.2,NaN,NaN,10.0,16.0,NaN,13.000000,2,13.000000
4,cccea85e,252.0,9.100159,50.758611,0.0,0.0,0.0,0.0,836.0,167.2,104.0,213.0,249.0,86.0,184.0,64.307692,13,29.857143


In [21]:
df_std_ingresos_diarios = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('d')]).size().reset_index(name='visitas_diarias')

In [22]:
df_std_ingresos_diarios = df_std_ingresos_diarios.groupby('person')['visitas_diarias'].std().fillna(0).reset_index(name='std_visitas_diarias')

In [23]:
df_std_ingresos_diarios.head()

,person,std_visitas_diarias
0,0008ed71,0.000000
1,00091926,21.868755
2,00091a7a,0.000000
3,000ba417,10.016653
4,000c79fe,0.000000


In [24]:
df_features_numericos = df_features_numericos.merge(df_std_ingresos_diarios,on='person',how='left')

In [25]:
df_features_numericos.head()

,person,Cantidad_visitas,Promedio_visitas_producto,horas_mirando_productos,visitas_mes1,visitas_mes2,visitas_mes3,visitas_mes4,visitas_mes5,promedio_ingreso_mensual,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5,promedio_registros_diarias,dias_ingresando,promedio_visitas_hora,std_visitas_diarias
0,4886f805,4.0,5.043919,0.315278,0.0,0.0,0.0,0.0,9.0,1.8,NaN,NaN,9.0,NaN,NaN,9.000000,1,9.000000,0.000000
1,ad93850f,17.0,9.904207,22.058056,0.0,0.0,0.0,0.0,65.0,13.0,NaN,13.0,45.0,7.0,NaN,16.250000,4,13.000000,17.651723
2,0297fc1e,213.0,9.325721,191.342500,120.0,48.0,87.0,139.0,173.0,113.4,67.0,99.0,151.0,200.0,50.0,9.610169,59,5.906250,6.158865
3,2d681dd8,6.0,8.860133,0.092222,0.0,0.0,0.0,0.0,26.0,5.2,NaN,NaN,10.0,16.0,NaN,13.000000,2,13.000000,4.242641
4,cccea85e,252.0,9.100159,50.758611,0.0,0.0,0.0,0.0,836.0,167.2,104.0,213.0,249.0,86.0,184.0,64.307692,13,29.857143,28.350146


In [26]:
df_std_ingresos_hora = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('h')]).size().reset_index(name='visitas_hora')

In [27]:
df_std_ingresos_hora = df_std_ingresos_hora.groupby('person')['visitas_hora'].std().fillna(0).reset_index(name='std_visitas_hora')

In [28]:
df_std_ingresos_hora.head()

,person,std_visitas_hora
0,0008ed71,1.000000
1,00091926,12.201511
2,00091a7a,0.000000
3,000ba417,22.662990
4,000c79fe,0.000000


In [29]:
df_features_numericos = df_features_numericos.merge(df_std_ingresos_hora, on='person', how='left')

In [30]:
df_features_numericos.head()

,person,Cantidad_visitas,Promedio_visitas_producto,horas_mirando_productos,visitas_mes1,visitas_mes2,visitas_mes3,visitas_mes4,visitas_mes5,promedio_ingreso_mensual,registros_semana_1,registros_semana_2,registros_semana_3,registros_semana_4,registros_semana_5,promedio_registros_diarias,dias_ingresando,promedio_visitas_hora,std_visitas_diarias,std_visitas_hora
0,4886f805,4.0,5.043919,0.315278,0.0,0.0,0.0,0.0,9.0,1.8,NaN,NaN,9.0,NaN,NaN,9.000000,1,9.000000,0.000000,0.000000
1,ad93850f,17.0,9.904207,22.058056,0.0,0.0,0.0,0.0,65.0,13.0,NaN,13.0,45.0,7.0,NaN,16.250000,4,13.000000,17.651723,10.770330
2,0297fc1e,213.0,9.325721,191.342500,120.0,48.0,87.0,139.0,173.0,113.4,67.0,99.0,151.0,200.0,50.0,9.610169,59,5.906250,6.158865,3.800493
3,2d681dd8,6.0,8.860133,0.092222,0.0,0.0,0.0,0.0,26.0,5.2,NaN,NaN,10.0,16.0,NaN,13.000000,2,13.000000,4.242641,4.242641
4,cccea85e,252.0,9.100159,50.758611,0.0,0.0,0.0,0.0,836.0,167.2,104.0,213.0,249.0,86.0,184.0,64.307692,13,29.857143,28.350146,22.130445


Max min y diferencia entre ellos sobre la cantidad de visitas diarias

In [31]:
df_max_ingresos_dia = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('d')]).size().reset_index(name='visitas_diarias')

In [32]:
df_max_ingresos_dia = df_max_ingresos_dia.groupby('person')['visitas_diarias'].max().reset_index(name='max_visitas_diarias')

In [33]:
df_features_numericos = df_features_numericos.merge(df_max_ingresos_dia, on='person', how='left')

In [34]:
df_min_ingresos_dia = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('d')]).size().reset_index(name='visitas_diarias')

In [35]:
df_min_ingresos_dia = df_min_ingresos_dia.groupby('person')['visitas_diarias'].min().reset_index(name='min_visitas_diarias')

In [36]:
df_features_numericos = df_features_numericos.merge(df_min_ingresos_dia, on='person', how='left')

In [37]:
df_features_numericos['diff_min_max_diaria'] = df_features_numericos['max_visitas_diarias'] - df_features_numericos['min_visitas_diarias'] 

Max min y diferencia entre ellos sobre la cantidad de visitas por hora

In [38]:
df_max_ingresos_hora = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('h')]).size().reset_index(name='visitas_hora')

In [39]:
df_max_ingresos_hora = df_max_ingresos_hora.groupby('person')['visitas_hora'].max().reset_index(name='max_visitas_hora')

In [40]:
df_features_numericos = df_features_numericos.merge(df_max_ingresos_hora, on='person', how='left')

In [41]:
df_min_ingresos_hora = df_ingresos_diarios.groupby(['person',df_ingresos_diarios['timestamp'].dt.floor('h')]).size().reset_index(name='visitas_hora')

In [42]:
df_min_ingresos_hora = df_min_ingresos_hora.groupby('person')['visitas_hora'].min().reset_index(name='min_visitas_hora')

In [43]:
df_features_numericos = df_features_numericos.merge(df_min_ingresos_hora, on='person', how='left')

In [44]:
df_features_numericos['diff_min_max_hora'] = df_features_numericos['max_visitas_hora'] - df_features_numericos['min_visitas_hora'] 

<hola>

Ver que dia realizo por primera y ultima vez el acceso a la pagina

In [45]:
df_ingresos_diarios['dia_del_año'] = df_ingresos_diarios['timestamp'].dt.dayofyear

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [46]:
df_dia_ultimo_ingreso = df_ingresos_diarios.groupby('person')['dia_del_año'].max().reset_index(name='ultimo_ingreso_dia')

In [47]:
df_features_numericos = df_features_numericos.merge(df_dia_ultimo_ingreso, on='person', how='left')

In [48]:
df_dia_primer_ingreso = df_ingresos_diarios.groupby('person')['dia_del_año'].min().reset_index(name='primer_ingreso_dia')

In [49]:
df_features_numericos = df_features_numericos.merge(df_dia_primer_ingreso, on='person', how='left')

In [50]:
df_features_numericos['diferencia_primer_ultimo_dia'] = df_features_numericos['ultimo_ingreso_dia'] - df_features_numericos['primer_ingreso_dia']

In [51]:

df_features_numericos.shape

(38829, 29)

Analizo la __primera quincena__ de los meses

In [52]:
df_temporal = df_ingresos_diarios

In [53]:
df_temporal['dia'] = df_temporal['timestamp'].dt.day

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
df_temporal['semana_del_dia'] = (df_temporal['dia']-1) // 7 + 1

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [55]:
df_temporal['quincena_numero'] = df_temporal['semana_del_dia'].apply(lambda x: 1 if(x==1 or x==2) else 2)

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
df_persona_actividad_quincena = df_temporal.groupby(['person','quincena_numero']).size()\
                                            .reset_index(name='registros_en_dicha_quincena')

In [57]:
df_persona_actividad_quincena_1 = df_persona_actividad_quincena.groupby('quincena_numero').get_group(1)\
                                                                .reset_index()

In [58]:
df_persona_actividad_quincena_2 = df_persona_actividad_quincena.groupby('quincena_numero').get_group(2)\
                                                                .reset_index()

In [59]:
df_persona_actividad_quincena_1 = df_persona_actividad_quincena_1.drop(columns=['index','quincena_numero'])
df_persona_actividad_quincena_1.columns = ['person','registros_primeras_quincenas']

In [60]:
df_persona_actividad_quincena_2 = df_persona_actividad_quincena_2.drop(columns=['index','quincena_numero'])
df_persona_actividad_quincena_2.columns = ['person','registros_segundas_quincenas']

In [61]:
df_personas = df_original['person'].drop_duplicates().to_frame()

In [62]:
df_actividad_por_quincena = df_personas.merge(df_persona_actividad_quincena_1, on='person', how='left')

In [63]:
df_actividad_por_quincena = df_actividad_por_quincena.merge(df_persona_actividad_quincena_2,\
                                                           on='person',\
                                                           how='left')

In [64]:
df_actividad_por_quincena = df_actividad_por_quincena.fillna(0)

In [65]:
df_actividad_por_quincena['mayor_quincena_1'] = df_actividad_por_quincena['registros_primeras_quincenas']\
                                                            > df_actividad_por_quincena['registros_segundas_quincenas']

In [66]:
df_actividad_por_quincena['mayor_quincena_1'] =df_actividad_por_quincena['mayor_quincena_1'].astype(int)

In [67]:
df_actividad_por_quincena.to_csv(path_or_buf = 'features_a_probar.csv', index = False)

# Features solo del 5to mes

In [68]:
df_5to_mes_only = df_original[['person','timestamp','event','model','country']]

In [69]:
df_5to_mes_only['mes'] = pd.to_datetime(df_5to_mes_only['timestamp']).dt.month

/home/amaherok/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [70]:
df_5to_mes_only = df_5to_mes_only.groupby('mes').get_group(5)

In [71]:
df_checkouts_5 = df_5to_mes_only.loc[df_5to_mes_only['event'] == 'checkout',:]

In [72]:
df_num_check_person = df_checkouts_5.groupby('person').size().reset_index(name='cant_checkouts_5')

In [73]:
df_features_mes_5 = df_personas.merge(df_num_check_person, on='person', how='left')

In [74]:
df_pct_checkouts = df_5to_mes_only.groupby('person')['event'].value_counts(normalize=True)\
                                                            .reset_index(name='%checkouts')

In [75]:
df_pct_checkouts = df_pct_checkouts.loc[df_pct_checkouts['event'] == 'checkout',:]

In [76]:
df_pct_checkouts = df_pct_checkouts.drop(columns=['event'])

In [77]:
df_pct_checkouts['%checkouts'] = df_pct_checkouts['%checkouts'] * 100 

In [78]:
df_features_mes_5 = df_features_mes_5.merge(df_pct_checkouts, on='person', how='left')

In [79]:
df_features_mes_5.shape

(38829, 3)

In [80]:
df_5to_mes_only['semana'] = pd.to_datetime(df_5to_mes_only['timestamp']).dt.week

In [81]:
df_5to_mes_ultima_semana = df_5to_mes_only.groupby('semana').get_group(22)

In [82]:
df_checkouts_ultima_semana = df_5to_mes_ultima_semana.groupby('person')['event'].value_counts()\
                                                    .reset_index(name='checkouts_ult_semana')

In [83]:
df_checkouts_ultima_semana = df_checkouts_ultima_semana.loc[df_checkouts_ultima_semana['event'] == 'checkout',:]

In [84]:
df_checkouts_ultima_semana = df_checkouts_ultima_semana.drop(columns=['event'])

In [85]:
df_features_mes_5 = df_features_mes_5.merge(df_checkouts_ultima_semana, on='person', how='left')

In [86]:
df_features_mes_5.shape

(38829, 4)

In [87]:
df_5to_mes_primera_semana = df_5to_mes_only.groupby('semana').get_group(18)

In [88]:
df_actividad_primera_semana = df_5to_mes_primera_semana.groupby('person').size()\
                                                    .reset_index(name='act_primera_semana')

In [89]:
df_actividad_ultima_semana = df_5to_mes_ultima_semana.groupby('person').size()\
                                                    .reset_index(name='act_ultima_semana')

In [90]:
df_relacion_primera_ultima = df_actividad_primera_semana.merge(df_actividad_ultima_semana, on='person', how='outer')

In [91]:
df_relacion_primera_ultima = df_relacion_primera_ultima.fillna(0)

In [92]:
df_relacion_primera_ultima['mayor_actividad_ult_semana'] = df_relacion_primera_ultima['act_ultima_semana'] >= df_relacion_primera_ultima['act_primera_semana'] 

In [93]:
df_features_mes_5 = df_features_mes_5.merge(df_relacion_primera_ultima, on='person', how='left')

In [95]:
df_cant_modelos = df_5to_mes_only.groupby('person')['model'].nunique().reset_index(name='cant_modelos_distintos')

In [96]:
df_cant_modelos.shape

(37143, 2)

In [97]:
df_features_mes_5 = df_features_mes_5.merge(df_cant_modelos, on='person', how='left')

In [99]:
df_cant_checkouts_dif_modelos = df_5to_mes_only.loc[df_5to_mes_only['event'] == 'checkout',:]

In [100]:
df_cant_checkouts_dif_modelos = df_cant_checkouts_dif_modelos.groupby('person')['model'].nunique()\
                                                            .reset_index(name='cant_checkouts_dif_modelos')

In [101]:
df_features_mes_5 = df_features_mes_5.merge(df_cant_checkouts_dif_modelos, on='person', how='left')

In [102]:
df_features_mes_5.shape

(38829, 9)

In [103]:
df_interaccion_modelo = df_5to_mes_only.groupby(['person','model']).size().reset_index(name='cant_interacciones_por_modelo')

In [104]:
df_datos_int_modelos = df_interaccion_modelo.groupby('person')\
.agg({'cant_interacciones_por_modelo' : ['max','mean']}).reset_index()

In [105]:
df_datos_int_modelos.columns = df_datos_int_modelos.columns.droplevel(0)

In [106]:
df_datos_int_modelos.columns = ['person','MAX(cant_interacciones_por_modelo)','MEAN(cant_interacciones_por_modelo)']

In [107]:
df_features_mes_5 = df_features_mes_5.merge(df_datos_int_modelos, on='person', how='left')

In [108]:
df_features_mes_5.shape

(38829, 11)

In [109]:
df_checkouts_modelo = df_checkouts_5.groupby(['person','model']).size().reset_index(name='cant_checkout_por_modelo')

In [110]:
df_checkouts_modelo.head()

,person,model,cant_checkout_por_modelo
0,0008ed71,LG G4 H818P,1
1,0008ed71,Samsung Galaxy S6 Flat,1
2,0008ed71,iPhone SE,1
3,00091926,iPhone 6S,2
4,000ba417,Motorola Moto G3 4G,1


In [111]:
df_features_mes_5.shape

(38829, 11)

In [112]:
def my_agg(x):
    names = {
        'Checkout max': x['cant_checkout_por_modelo'].max(),
        'Checkout mean': x['cant_checkout_por_modelo'].mean()
    }
    return pd.Series(names, index=['Checkout max','Checkout mean'])

In [113]:
df_checkouts_modelo = df_checkouts_modelo.groupby('person').apply(my_agg)

In [114]:
df_features_mes_5 = df_features_mes_5.merge(df_checkouts_modelo, on='person', how='left')

In [115]:
df_features_mes_5 = df_features_mes_5.fillna(0)

In [116]:
df_features_mes_5['mayor_actividad_ult_semana'] = df_features_mes_5['mayor_actividad_ult_semana'].astype(int)

In [117]:
df_features_mes_5.shape

(38829, 13)

In [118]:
df_5to_mes_only_lead = df_5to_mes_only.groupby('event').get_group('lead')

In [119]:
df_5to_mes_only_lead_cantidad = df_5to_mes_only_lead.groupby('person').size().reset_index(name='cant_lead_5to_mes')

In [120]:
df_5to_mes_only_lead_cantidad.shape

(239, 2)

In [121]:
df_features_mes_5 = df_features_mes_5.merge(df_5to_mes_only_lead_cantidad, on='person', how='left')

In [122]:
df_features_mes_5 = df_features_mes_5.fillna(0)

In [123]:
df_5to_mes_lead_cantidad_modelos_consultados = df_5to_mes_only_lead.groupby('person')['model'].nunique()\
                                                                .reset_index(name='cant_modelos_que_consulto_stock')

In [124]:
df_features_mes_5 = df_features_mes_5.merge(df_5to_mes_lead_cantidad_modelos_consultados, on='person',how='left')

In [125]:
df_features_mes_5 = df_features_mes_5.fillna(0)

In [126]:
df_features_mes_5.to_csv(path_or_buf = 'features_numericos_mes_5.csv', index = False)

In [127]:
df_features_mes_5.shape

(38829, 15)

In [128]:
df_features_numericos.shape

(38829, 29)

# Merge features mes 5 con features de Nacho

In [129]:
df_nacho = pd.read_csv('features_nacho.csv', low_memory=False)

In [130]:
df_nacho.columns

Index(['person', 'conversion_sum', 'conversion_mean', 'viewed_sum',
       'viewed_mean', 'viewed_std', 'dif_5_check', 'last_day_check',
       'first_day_check', 'last_week_check', 'first_week_check'],
      dtype='object')

In [131]:
df_features_mes_5 = pd.read_csv('features_numericos_mes_5.csv', low_memory=False)

In [132]:
df_features_mes_5.columns

Index(['person', 'cant_checkouts_5', '%checkouts', 'checkouts_ult_semana',
       'act_primera_semana', 'act_ultima_semana', 'mayor_actividad_ult_semana',
       'cant_modelos_distintos', 'cant_checkouts_dif_modelos',
       'MAX(cant_interacciones_por_modelo)',
       'MEAN(cant_interacciones_por_modelo)', 'Checkout max', 'Checkout mean',
       'cant_lead_5to_mes', 'cant_modelos_que_consulto_stock'],
      dtype='object')

In [133]:
df_features_numericos = df_nacho.merge(df_features_mes_5, on='person', how='outer')

In [134]:
df_features_numericos.columns

Index(['person', 'conversion_sum', 'conversion_mean', 'viewed_sum',
       'viewed_mean', 'viewed_std', 'dif_5_check', 'last_day_check',
       'first_day_check', 'last_week_check', 'first_week_check',
       'cant_checkouts_5', '%checkouts', 'checkouts_ult_semana',
       'act_primera_semana', 'act_ultima_semana', 'mayor_actividad_ult_semana',
       'cant_modelos_distintos', 'cant_checkouts_dif_modelos',
       'MAX(cant_interacciones_por_modelo)',
       'MEAN(cant_interacciones_por_modelo)', 'Checkout max', 'Checkout mean',
       'cant_lead_5to_mes', 'cant_modelos_que_consulto_stock'],
      dtype='object')

In [135]:
df_features_numericos.head()

,person,conversion_sum,conversion_mean,viewed_sum,viewed_mean,viewed_std,dif_5_check,last_day_check,first_day_check,last_week_check,...,act_ultima_semana,mayor_actividad_ult_semana,cant_modelos_distintos,cant_checkouts_dif_modelos,MAX(cant_interacciones_por_modelo),MEAN(cant_interacciones_por_modelo),Checkout max,Checkout mean,cant_lead_5to_mes,cant_modelos_que_consulto_stock
0,4886f805,0.0,0.0,4.0,NaN,4.000000,0.0,18.0,18.0,4.0,...,0.0,0,1.0,1.0,5.0,5.000000,1.0,1.0,0.0,0.0
1,ad93850f,0.0,0.0,20.0,7.371115,6.666667,0.0,14.0,14.0,0.0,...,0.0,0,3.0,1.0,18.0,7.000000,1.0,1.0,0.0,0.0
2,0297fc1e,0.0,0.0,404.0,5.228295,7.214286,12.0,22.0,10.0,1.0,...,8.0,0,11.0,2.0,79.0,12.272727,1.0,1.0,0.0,0.0
3,2d681dd8,0.0,0.0,13.0,0.707107,6.500000,0.0,27.0,27.0,6.0,...,0.0,0,3.0,1.0,6.0,4.666667,1.0,1.0,0.0,0.0
4,cccea85e,0.0,0.0,739.0,26.241971,56.846154,0.0,11.0,11.0,4.0,...,252.0,1,57.0,1.0,253.0,12.982456,1.0,1.0,0.0,0.0


Features numericos que saque antes, que no son solo del mes 5

In [1027]:
df_features_numericos_viejos = pd.read_csv('features_numericos.csv', low_memory=False)

In [1028]:
df_features_numericos_viejos = df_features_numericos_viejos[['person', 'Cantidad_visitas', 'Promedio_visitas_producto',\
       'horas_mirando_productos', 'visitas_mes1', 'visitas_mes2',\
       'visitas_mes3', 'visitas_mes4', 'visitas_mes5',\
       'promedio_ingreso_mensual']]

In [1029]:
df_features_numericos_viejos.columns

Index(['person', 'Cantidad_visitas', 'Promedio_visitas_producto',
       'horas_mirando_productos', 'visitas_mes1', 'visitas_mes2',
       'visitas_mes3', 'visitas_mes4', 'visitas_mes5',
       'promedio_ingreso_mensual'],
      dtype='object')

In [1030]:
df_features_numericos = df_features_numericos.merge(df_features_numericos_viejos, on='person', how='left')

In [1031]:
df_features_numericos = df_features_numericos.fillna(0) 

In [1032]:
df_features_numericos.shape

(38829, 34)

In [ ]:
df_features_numericos.to_csv(path_or_buf = 'features_numericos_merge_final.csv',index = False)